In [1]:
import os, requests, zipfile, io, re, pandas, dotenv

dotenv.load_dotenv('../.env')

api = os.getenv('MAGIC_API') or 'https://api.earthref.org/v1/MagIC/{}'
print(api)

https://api.earthref.org/v1/MagIC/{}


### Get the most recently updated 10 contributions IDs that have vgp_lat and vgp_lon data

In [2]:
# Get the matching sites rows into a Pandas dataframe
response = requests.get(api.format('search/contributions'), params={'included_columns': ['vgp_lat', 'vgp_lon'], 'n_max_rows': 10})
print(response.request.method, response.request.url, response.status_code)
if (response.status_code == 200):
    print('Total number of matching contributions:', response.json()['total'])
    print([x['id'] for x in response.json()['results']])
elif (response.status_code == 204):
    print('No contributions in MagIC match the search parameters.', '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

GET https://api.earthref.org/v1/MagIC/search/contributions?included_columns=vgp_lat&included_columns=vgp_lon&n_max_rows=10 200
Total number of matching contributions: 1323
[19458, 19464, 19463, 19460, 19452, 19442, 19436, 19313, 19422, 19414]


### Given a MagIC contribution ID, download the MagIC contribution text file if it is public

In [3]:
# Try to download the contribution text for an invalid ID
response = requests.get(api.format('data'), headers={'Accept': 'application/json'}, params={'id': 'a'})
print(response.request.method, response.request.url, response.status_code)
print('Error:', response.json()['errors'][0]['message'], '\n')

# Try to download the contribution text for what could be valid, but are nonexistant, IDs
response = requests.get(api.format('data'), headers={'Accept': 'application/json'}, params={'id': [1000, 1001]})
print(response.request.method, response.request.url, response.status_code)
if (response.status_code == 204):
    print('Public contribution IDs 1000 and 1001 are not found in MagIC', '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Download the contribution text for a valid ID and print the first 200 characters
contribution_id = 16761
response = requests.get(api.format('data'), headers={'Accept': 'text/plain'}, params={'id': contribution_id})
print(response.request.method, response.request.url, response.status_code)
if (response.status_code == 200):
    contribution_text = response.text
    print(contribution_text[0:200], '\n')
elif (response.status_code == 204):
    print('Public contribution ID = \'{}\' not found in MagIC'.format(contribution_id), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Get the ID for the latest public MagIC contribution
latest_contribution_id_response = requests.get(api.format('search/contributions?n_max_rows=1'))
print(latest_contribution_id_response.request.method, latest_contribution_id_response.request.url, latest_contribution_id_response.status_code)
if (latest_contribution_id_response.status_code == 200):
    latest_contribution_id = latest_contribution_id_response.json()['results'][0]['id']
elif (latest_contribution_id_response.status_code == 204):
    print('Latest contribution ID not retrieved in MagIC', '\n')
else:
    print('Error:', latest_contribution_id_response.json()['errors'][0]['message'], '\n')

# Get the contribution text for the ID and print the first 200 characters
latest_contribution_response = requests.get(api.format('download'), headers={'Accept': 'application/zip'}, params={'id': latest_contribution_id})
print(latest_contribution_response.request.method, latest_contribution_response.request.url, latest_contribution_response.status_code)
if (latest_contribution_response.status_code == 200):
    latest_contribution_zip = zipfile.ZipFile(io.BytesIO(latest_contribution_response.content))
    latest_contribution_text = io.TextIOWrapper(latest_contribution_zip.open('{}/magic_contribution_{}.txt'.format(latest_contribution_id, latest_contribution_id))).read()
    print(latest_contribution_text[0:200], '\n')
elif (latest_contribution_response.status_code == 204):
    print('Public contribution ID = \'{}\' not found in MagIC'.format(latest_contribution_id), '\n')
else:
    print('Error:', latest_contribution_response.json()['errors'][0]['message'], '\n')

GET https://api.earthref.org/v1/MagIC/data?id=a 400
Error: must be integer 

GET https://api.earthref.org/v1/MagIC/data?id=1000&id=1001 400
Error: must be integer 

GET https://api.earthref.org/v1/MagIC/data?id=16761 200
tab delimited	contribution
id	version	timestamp	contributor	data_model_version	reference
16761	2	2020-02-23T23:03:45.034Z	@ltauxe	3.0	10.1016/J.EPSL.2014.12.034
>>>>>>>>>>
tab delimited	locations
loca 

GET https://api.earthref.org/v1/MagIC/search/contributions?n_max_rows=1 200
GET https://api.earthref.org/v1/MagIC/download?id=19409 500
Error: Failed to retrieve contributions [19409] for download. 



### Given a DOI, download the MagIC contribution text file for the latest public version(s)

In [4]:
# Download and extract the latest contribution file to a magic_contribution.txt file
reference_doi = '10.1016/J.PEPI.2017.05.009'
response = requests.get(api.format('download'), headers={'Accept': 'application/zip'}, params={'doi': reference_doi, 'n_max_contributions': 1})
print(response.request.method, response.request.url, response.status_code)
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    for filename in contribution_zip.namelist():
        if (re.match(r'^\d+\/magic_contribution_\d+\.txt', filename)):
            contribution_text = io.TextIOWrapper(contribution_zip.open(filename)).read()
            with open('downloads/magic_contribution.txt', 'wt') as fh:
                fh.write(contribution_text)
            print(filename, 'extracted to downloads/magic_contribution.txt', '\n')
elif (response.status_code == 204):
    print('Public contribution with a reference DOI = \'{}\' not found in MagIC'.format(reference_doi), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Get the contribution text for each matching contribution and print the first 200 characters
reference_doi = '10.1016/J.EPSL.2014.12.034'
response = requests.get(api.format('download'), headers={'Accept': 'application/zip'}, params={'doi': reference_doi, 'n_max_contributions': 2})
print(response.request.method, response.request.url, response.status_code)
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    for filename in contribution_zip.namelist():
        if (re.match(r'^\d+\/magic_contribution_\d+\.txt', filename)):
            contribution_text = io.TextIOWrapper(contribution_zip.open(filename)).read()
            print(contribution_text[0:200], '\n')
elif (response.status_code == 204):
    print('Public contributions with a reference DOI = \'{}\' not found in MagIC'.format(reference_doi), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

GET https://api.earthref.org/v1/MagIC/download?doi=10.1016%2FJ.PEPI.2017.05.009&n_max_contributions=1 500
Error: Failed to retrieve contributions [16814] for download. 

GET https://api.earthref.org/v1/MagIC/download?doi=10.1016%2FJ.EPSL.2014.12.034&n_max_contributions=2 500
Error: Failed to retrieve contributions [16761, 16760] for download. 



### Download a contribution file and validate it

In [5]:
contribution_id = 16901

response = requests.get(api.format('data'), params={'id': contribution_id})
print(response.request.method, response.request.url, response.status_code)
if (response.status_code == 200):
    contribution_file = 'downloads/magic_contribution_{}.txt'.format(contribution_id)
    open(contribution_file, 'w').write(response.text)
    print('Retrieved contribution data with ID', contribution_id, '\n')
    
    with open(contribution_file, 'rb') as f:
        validation_response = requests.post(api.format('validate'),
            headers={'Content-Type': 'text/plain'}, 
            data=f
        )

    print(validation_response.request.method, validation_response.request.url)
    if (validation_response.status_code == 200):
        validation_results = validation_response.json()['validation']
        print('Validated contribution with ID', contribution_id, ':\n', validation_results)
    else:
        print('Error Validating a Private Contribution:', validation_response.json()['errors'][0]['message'], '\n')
else:
    print('Retrieve Public Contribution Error:', response.json()['errors'][0]['message'], '\n')

GET https://api.earthref.org/v1/MagIC/data?id=16901 200
Retrieved contribution data with ID 16901 

POST https://api.earthref.org/v1/MagIC/validate
Validated contribution with ID 16901 :
 {'errors': [{'table': 'contribution', 'column': 'lab_names', 'message': 'The contribution table is missing required column "lab_names".', 'rows': [1]}], 'warnings': []}


### Get the 50 latest public sites in MagIC that mention basalt or sandstone

In [6]:
# Get the matching sites rows into a Pandas dataframe
response = requests.get(api.format('search/sites'), params={'query': 'basalt OR sandstone', 'n_max_rows': 50})
print(response.request.method, response.request.url, response.status_code)
if (response.status_code == 200):
    sites = response.json()['results']
elif (response.status_code == 204):
    print('No sites in MagIC match the search parameters.', '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

sites_df = pandas.DataFrame(sites)
sites_df.head(50)

GET https://api.earthref.org/v1/MagIC/search/sites?query=basalt+OR+sandstone&n_max_rows=50 200


,external_database_ids,result_type,description,lon,geologic_types,int_abs_sigma,method_codes,vdm_sigma,geologic_classes,int_abs,...,bed_dip,dir_alpha95,specimens,dir_n_specimens_planes,samples,age_low,dir_r,dir_tilt_correction,dir_nrm_origin,result_quality
0,PINT[8044],i,FL8,340,Lava Flow,5.55E-06,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,1.45E+22,Extrusive,1.84E-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PINT[8034],i,HU4B,340,Lava Flow,2.27E-06,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,3.20E+21,Extrusive,1.51E-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PINT[8042],i,PV'3,340,Lava Flow,1.97E-06,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,4.90E+21,Extrusive,8.72E-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PINT[8037],i,PV5,340,Lava Flow,1.12E-05,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,3.40E+21,Extrusive,5.49E-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PINT[8038],i,PV8,340,Lava Flow,1.34E-06,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,,Extrusive,1.01E-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PINT[8039],i,PV9,340,Lava Flow,2.25E-06,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,5.50E+21,Extrusive,8.32E-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,PINT[8031],i,S02,340,Lava Flow,,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,,Extrusive,6.11E-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,PINT[8030],i,U14,340,Lava Flow,,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,,Extrusive,6.19E-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PINT[8032],i,U16,340,Lava Flow,,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,,Extrusive,1.00E-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PINT[8033],i,U17,340,Lava Flow,,LP-PI-ALT-PMRM : LP-PI-TRM-ZI:GM-NO,,Extrusive,2.28E-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
